In [1]:
# Install a Drive FUSE wrapper
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build3) ...
Setting up python-software-properties (0.96.24.17) ...
Setting up software-properties-common (0.96.24.17) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Processing triggers for dbus (1.10.22-1ubuntu1) ...
gpg: keybox '/tmp/tmp2bumikut/pubring.g

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/

Files in Drive:
acc details.txt  csgo config backup  GCollab		  others
clg id		 Database	     image search engine
college		 Deep learning       internship 
computer vision  eyrc temp	     machine learning


In [5]:
import keras

Using TensorFlow backend.


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense,Flatten,GlobalAveragePooling2D

In [0]:
wt_notop ='drive/GCollab/urban_rural/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
model = Sequential()
model.add(ResNet50(include_top=False,weights=wt_notop, pooling='avg'))
model.add(Dense(32,activation='relu')) ## added for improve model
model.add(Dense(2,activation='softmax'))


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 32)                65568     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 23,653,346
Trainable params: 23,600,226
Non-trainable params: 53,120
_________________________________________________________________


In [0]:
# first layer is already trained.
model.layers[0].trainable = False

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
train_path = 'drive/GCollab/urban_rural/urban_rural_images/train'
valid_path = 'drive/GCollab/urban_rural/urban_rural_images/valid'

**WITHOUT AUGMENTATION**

In [12]:
# the number of samples processed for each epoch is batch_size * steps_per_epochs.
# 72 trainig images(36*2)
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator()

train_generator = data_generator.flow_from_directory(
        train_path,
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        valid_path,
        target_size=(image_size, image_size),
        class_mode='categorical')

model.fit_generator(
        train_generator,
        epochs=5,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=1)

Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/5
3/3 [==============================] - 76s 25s/step - loss: 0.5405 - acc: 0.7361 - val_loss: 0.8993 - val_acc: 0.5000
Epoch 2/5
3/3 [==============================] - 1s 395ms/step - loss: 0.3477 - acc: 0.8750 - val_loss: 0.5850 - val_acc: 0.7000
Epoch 3/5
3/3 [==============================] - 1s 466ms/step - loss: 0.3453 - acc: 0.8889 - val_loss: 0.4458 - val_acc: 0.8000
Epoch 4/5
3/3 [==============================] - 2s 531ms/step - loss: 0.1620 - acc: 0.9861 - val_loss: 0.4162 - val_acc: 0.8000
Epoch 5/5
3/3 [==============================] - 2s 503ms/step - loss: 0.1241 - acc: 0.9861 - val_loss: 0.3905 - val_acc: 0.8000


**WITH AUGMENTATION**

In [13]:
image_size = 224
data_generator_with_aug = ImageDataGenerator(
                                   horizontal_flip=True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

train_generator = data_generator_with_aug.flow_from_directory(
        train_path,
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

data_generator_no_aug = ImageDataGenerator()  # We dont apply augmentation in valid.set

validation_generator = data_generator_no_aug.flow_from_directory(
        valid_path,
        target_size=(image_size, image_size),
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=2,
        validation_data=validation_generator,
        validation_steps=1)

Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/2
3/3 [==============================] - 3s 875ms/step - loss: 0.2029 - acc: 0.9444 - val_loss: 0.3385 - val_acc: 0.8000
Epoch 2/2
3/3 [==============================] - 3s 943ms/step - loss: 0.1551 - acc: 0.9444 - val_loss: 0.3204 - val_acc: 0.8000


**IMPROVING ACCURACY**

In [15]:
image_size = 224


data_generator_with_aug = ImageDataGenerator(
                                              horizontal_flip = True,
                                              width_shift_range =0.4,
                                              height_shift_range =0.4,
                                              rescale=1./255,
                                              zoom_range=0.2)
            

data_generator_no_aug = ImageDataGenerator()



train_generator = data_generator_with_aug.flow_from_directory(
        directory = train_path,
        target_size=(image_size, image_size),
        batch_size=8,
        class_mode='categorical')


validation_generator = data_generator_no_aug.flow_from_directory(
        directory = valid_path,
        target_size=(image_size, image_size),
        class_mode='categorical')

model.fit_generator(
        train_generator, # specify where model gets training data
        epochs = 3,
        steps_per_epoch=25,
        validation_data=validation_generator) # specify where model gets training data

Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/3
25/25 [==============================] - 6s 251ms/step - loss: 0.2339 - acc: 0.9300 - val_loss: 0.6136 - val_acc: 0.8500
Epoch 2/3
25/25 [==============================] - 6s 243ms/step - loss: 0.1800 - acc: 0.9300 - val_loss: 0.2584 - val_acc: 0.9000
Epoch 3/3
25/25 [==============================] - 6s 246ms/step - loss: 0.0819 - acc: 0.9850 - val_loss: 0.3632 - val_acc: 0.9000


In [25]:
image_size = 224


data_generator_with_aug = ImageDataGenerator(
                                              horizontal_flip = True,
                                              width_shift_range =0.3,
                                              height_shift_range =0.3)
            

data_generator_no_aug = ImageDataGenerator()



train_generator = data_generator_with_aug.flow_from_directory(
        directory = train_path,
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')


validation_generator = data_generator_no_aug.flow_from_directory(
        directory = valid_path,
        target_size=(image_size, image_size),
        class_mode='categorical')

model.fit_generator(
        train_generator, # specify where model gets training data
        epochs = 3,
        steps_per_epoch=19,
        validation_data=validation_generator) # specify where model gets training data

Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/3
19/19 [==============================] - 7s 343ms/step - loss: 0.0441 - acc: 0.9912 - val_loss: 0.3487 - val_acc: 0.9000
Epoch 2/3
19/19 [==============================] - 7s 345ms/step - loss: 0.0633 - acc: 0.9781 - val_loss: 0.3757 - val_acc: 0.9000
Epoch 3/3
19/19 [==============================] - 6s 338ms/step - loss: 0.0219 - acc: 1.0000 - val_loss: 0.3722 - val_acc: 0.9000


In [0]:
from keras.models import load_model
model.save('drive/GCollab/urban_rural/my_model.h5')

**PREDICTION**

In [0]:
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.preprocessing import image
import numpy as np

In [0]:
def prep_img(img_paths):
    
    imgs = [image.load_img(img,target_size=(224,224)) for img in img_paths]   #target size is imp!
    img_arr = np.array( [image.img_to_array(img) for img in imgs] )
    # no need to us expand array 
    preprocessed = preprocess_input(img_arr)
    return preprocessed

In [0]:
import os
img_dir = 'drive/GCollab/urban_rural/urban_rural_images/test'
filenames = ['test1.jpg','test2.jpg']
img_paths = [os.path.join(img_dir,file) for file in filenames]

In [0]:
data = prep_img(img_paths)
pred = model.predict(data)

In [20]:
pred

array([[6.1028624e-01, 3.8971376e-01],
       [2.3872595e-05, 9.9997616e-01]], dtype=float32)

In [21]:
train_generator.class_indices

{'rural': 0, 'urban': 1}


*   **rural:** 10
*   **urban** 01




In [0]:
pred = (pred>0.5) #set thres 

In [23]:
pred

array([[ True, False],
       [False,  True]])

In [24]:
i=1
for p in pred:
  
  if p[0]==True:
    print('test image '+str(i)+' is rural')
  if p[1]==True:
    print('test image '+str(i)+' is urban')
  i = i+1

test image 1 is rural
test image 2 is urban
